In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from tensorflow.keras.models import load_model

def readData(filePath):
    # attributes of the dataset
    columnNames = ['user_id','activity','timestamp','x-axis','y-axis','z-axis','x-rotate','y-rotate','z-rotate','arms','rrms','roll','pitch']
    data = pd.read_csv(filePath,header = None, names=columnNames,na_values=';')
    return data

def windows(data,size):
    start = 0
    while start< data.count():
        yield int(start), int(start + size)
        start+= (size/2)
# segmenting the time series
def segment_signal(data, window_size = 30):
    segments = np.empty((0,window_size,10))
    labels= np.empty((0))
    for (start, end) in windows(data['timestamp'],window_size):
        x = data['x-axis'][start:end]
        y = data['y-axis'][start:end]
        z = data['z-axis'][start:end]
        a = data['x-rotate'][start:end]
        b = data['y-rotate'][start:end]
        c = data['z-rotate'][start:end]
        d = data['arms'][start:end]
        e = data['rrms'][start:end]
        f = data['roll'][start:end]
        g = data['pitch'][start:end]
        if(len(data['timestamp'][start:end])==window_size):
            segments = np.vstack([segments,np.dstack([x,y,z,a,b,c,d,e,f,g])])
            labels = np.append(labels,stats.mode(data['activity'][start:end])[0][0])
    return segments, labels

dataset = readData('./TEST_DATA/test_total_added_sidecrunch.txt')
segments, labels = segment_signal(dataset) 

labels = np.asarray(pd.get_dummies(labels),dtype = np.int8) ##

numOfRows = segments.shape[1]
numOfColumns = segments.shape[2]
reshapedSegments = segments.reshape(segments.shape[0], numOfRows, numOfColumns,1)
testX = reshapedSegments
testX = np.nan_to_num(testX)
testY = labels

np.save('groundTruth.npy',testY)
np.save('testData.npy',testX)